In [109]:
from pathlib import Path
import pandas as pd

In [110]:
def dataframe_to_markdown(df):
    headers = list(df.columns)
    markdown = "| " + " | ".join(headers) + " |\n"
    markdown += "| " + " | ".join(["---"] * len(headers)) + " |\n"

    for _, row in df.iterrows():
        markdown += "| " + " | ".join(str(cell) for cell in row) + " |\n"
    return markdown

In [111]:
dataset = "LawDataset"
method = "CCHVAE"
model_name = "MultilayerPerceptron"
# model_name = "MultinomialLogisticRegression"

root = f"../models/{dataset}/{method}"

In [112]:
metrics = []
time_files = []

In [113]:
for i in range(5):
    path = Path(root) / f"fold_{i}" / f"cf_metrics_{model_name}.csv"
    df = pd.read_csv(path)
    metrics.append(df)

In [114]:
merged_df = pd.concat(metrics, axis=0, ignore_index=True)

In [115]:
mean_ = merged_df.mean(axis=0)
std_ = merged_df.std(axis=0)

In [116]:
def format_mean_std(m, s):
    return f"{m:.2f} ± {s:.2f}"


# Create the merged table
formatted = pd.DataFrame(
    {col: [format_mean_std(mean_[col], std_[col])] for col in merged_df.columns}
).T

In [117]:
used_metrics = [
    "validity",
    "prob_plausibility",
    "lof_scores_cf",
    "isolation_forest_scores_cf",
    "log_density_cf",
    "proximity_continuous_manhattan",
    "proximity_continuous_euclidean",
    "cf_search_time",
]

In [118]:
transposed = formatted.T
transposed = transposed[used_metrics]

In [119]:
transposed

,validity,prob_plausibility,lof_scores_cf,isolation_forest_scores_cf,log_density_cf,proximity_continuous_manhattan,proximity_continuous_euclidean,cf_search_time
0,0.78 ± 0.02,0.97 ± 0.02,3.26 ± 1.64,0.11 ± 0.01,-10.35 ± 0.89,0.73 ± 0.03,0.69 ± 0.03,6.65 ± 0.70


In [120]:
print(dataframe_to_markdown(transposed))

| validity | prob_plausibility | lof_scores_cf | isolation_forest_scores_cf | log_density_cf | proximity_continuous_manhattan | proximity_continuous_euclidean | cf_search_time |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 0.78 ± 0.02 | 0.97 ± 0.02 | 3.26 ± 1.64 | 0.11 ± 0.01 | -10.35 ± 0.89 | 0.73 ± 0.03 | 0.69 ± 0.03 | 6.65 ± 0.70 |



German

MLR
| validity |
| --- |
| 0.38 ± 0.06 |

MLP

| validity |
| --- |
| 0.57 ± 0.16 |

Law

MLR
| validity |
| --- |
| 0.89 ± 0.25 |

MLP

| validity |
| --- |
| 1.00 ± 0.00 |

Adult

MLR

| validity |
| --- |
| 0.57 ± 0.04 |

MLP

| validity |
| --- |
| 0.63 ± 0.06 |